In [ ]:
# 전이 학습 : 파인 튜닝

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch
from torch.utils.data import Dataset

# =============================
# 0. 기본 설정
# =============================
model_name = "skt/kogpt2-base-v2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =============================
# 1. 전이 학습용 데이터셋 정의
# =============================
class ChatDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.input_ids = []
        for txt in texts:
            encodings = tokenizer(txt, truncation=True, max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings["input_ids"]))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return {"input_ids": self.input_ids[idx], "labels": self.input_ids[idx]}

# 간단 예시 데이터 (실제 학습 데이터로 교체 가능) 
train_texts = [
    "User: 안녕하세요\nBot: 안녕하세요! 만나서 반갑습니다.\n",
    "User: 오늘 날씨 어때?\nBot: 오늘은 맑고 따뜻합니다.\n",
]

# 말뭉치를 이용해서 파인튜닝을 하는 경우 아래와 같이 불러오면 된다.
# 한국어 학습자 말뭉치 나눔터(https://kcorpus.korean.go.kr)
# aihub ( https://www.aihub.or.kr)

# with open("korpus_data.txt", "r", encoding="utf-8") as f:
#    train_texts = f.read().splitlines()


# =============================
# 2. 토크나이저와 모델 불러오기
# =============================
tokenizer = AutoTokenizer.from_pretrained(model_name)

# KoGPT2에는 pad_token이 없으므로 eos_token으로 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 모델을 불러오는 부분
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# =============================
# 3. 데이터셋 준비
# =============================
dataset = ChatDataset(train_texts, tokenizer)

# =============================
# 4. 전이 학습 설정
# =============================
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./kogpt2_finetuned", # 학습 결과(모델 가중치, 체크포인트, 로그 파일 등)가 저장되는 폴더 경로
    num_train_epochs=3,  # 전체 데이터셋을 몇 번 반복(에폭)해서 학습할지 결정
    per_device_train_batch_size=1, # GPU 하나당 배치 크기
    save_steps=100, # 100 스텝마다(100번 학습할 때마다) 모델을 저장
    save_total_limit=2, # 저장할 체크포인트 개수 제한, 가장 최근 2개의 체크포인트만 남기고 나머지는 자동으로 삭제
    logging_steps=10, # 10이면 10번 학습할 때마다 loss 등의 학습 로그가 나옴
    learning_rate=5e-5, # 모델 업데이트 속도(learning rate)
    report_to="none"  # 이 부분을 추가합니다.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# =============================
# 5. 전이 학습 수행 및 저장
# =============================
trainer.train()
trainer.save_model("./kogpt2_finetuned")
tokenizer.save_pretrained("./kogpt2_finetuned")
print("모델 전이 학습 완료 및 저장됨.")

In [ ]:
# 전이 학습한 모델을 불러와 실행한 후 학습후 저장

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# =============================
# 0. 모델 불러오기
# =============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained("./kogpt2_finetuned").to(device)
tokenizer = AutoTokenizer.from_pretrained("./kogpt2_finetuned")

# =============================
# 1. 챗봇 실행 함수
# =============================
def chat():
    print("=== 한국어 GPT-2 대화형 챗봇 (전이 학습 버전) ===")
    print("종료하려면 'quit' 입력\n")

    chat_history_ids = None
    max_history_tokens = 512

    while True:
        user_input = input("User: ")
        if user_input.lower() == "quit":
            print("Bot: 대화를 종료합니다. 안녕히 가세요!")
            break

        new_input_ids = tokenizer(f"User: {user_input}\nBot:", return_tensors="pt").input_ids.to(device)

        if chat_history_ids is None:
            chat_history_ids = new_input_ids
        else:
            total_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
            chat_history_ids = total_ids[:, -max_history_tokens:]

        outputs = model.generate(
            input_ids=chat_history_ids,
            max_length=len(chat_history_ids[0]) + 100,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_p=0.9,
            temperature=0.8,
            repetition_penalty=1.2
        )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        bot_reply = response.split("Bot:")[-1].split("User:")[0].strip()

        print(f"Bot: {bot_reply}")

        # 다음 턴을 위해 대화 기록 업데이트
        chat_history_ids = tokenizer(response, return_tensors="pt").input_ids.to(device)

# =============================
# 2. 챗봇 실행
# =============================
if __name__ == "__main__":
    chat()
